# DibParser (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fparsec/2.0.0-beta2/lib/netstandard2.1/FParsec.dll"
#r @"../../../../../../../.nuget/packages/fparsec/2.0.0-beta2/lib/netstandard2.1/FParsecCS.dll"

In [ ]:
#!import ../nbs/Common.fs

In [ ]:
open Common
open FParsec

## escapeCell (test)

In [ ]:
//// test

let inline escapeCell input =
    input
    |> String.split [| '\n' |]
    |> Array.map (function
        | line when line |> String.startsWith "\\#!" || line |> String.startsWith "\\#r" ->
            System.Text.RegularExpressions.Regex.Replace (line, "^\\\\#", "#")
        | line -> line
    )
    |> String.concat "\n"

In [ ]:
//// test

$"a{nl}\\#!magic{nl}b{nl}"
|> escapeCell
|> _equal (
    $"a{nl}#!magic{nl}b{nl}"
)

a
#!magic
b



In [ ]:
type Block =
    {
        magic : string
        content : string
    }

## magicMarker

In [ ]:
let magicMarker : Parser<string, unit> = pstring "#!"

In [ ]:
//// test

"#!magic"
|> run magicMarker
|> _equal (
    Success ("#!", (), Position ("", 2, 1, 3))
)

Success
      Item1: #!
      Item2: <null>
      Item3: Position
        Index: 2
        Line: 1
        Column: 3
        StreamName: 


In [ ]:
//// test

"##!magic"
|> run magicMarker
|> _equal (
    Failure (
        $"Error in Ln: 1 Col: 1{nl}##!magic{nl}^{nl}Expecting: '#!'{nl}",
        ParserError (
            Position ("", 0, 1, 1),
            (),
            ErrorMessageList (ExpectedString "#!")
        ),
        ()
    )
)

Failure
      Item1: Error in Ln: 1 Col: 1
##!magic
^
Expecting: '#!'

      Item2: ParserError
        Position: Position
          Index: 0
          Line: 1
          Column: 1
          StreamName: 
        UserState: <null>
        Messages: ErrorMessageList
          Head: ExpectedString
            String: #!
            Type: ExpectedString
          Tail: <null>
      Item3: <null>


## magicCommand

In [ ]:
let magicCommand =
    magicMarker
    >>. manyTill anyChar newline
    |>> (String.Concat >> String.trim)

In [ ]:
//// test

"""#!magic

a"""
|> run magicCommand
|> _equal (
    Success ("magic", (), Position ("", 8, 2, 1))
)

Success
      Item1: magic
      Item2: <null>
      Item3: Position
        Index: 8
        Line: 2
        Column: 1
        StreamName: 


In [ ]:
//// test

""" #!magic

a"""
|> run magicCommand
|> _equal (
    Failure (
        $"Error in Ln: 1 Col: 1{nl} #!magic{nl}^{nl}Expecting: '#!'{nl}",
        ParserError (
            Position ("", 0, 1, 1),
            (),
            ErrorMessageList (ExpectedString "#!")
        ),
        ()
    )
)

Failure
      Item1: Error in Ln: 1 Col: 1
 #!magic
^
Expecting: '#!'

      Item2: ParserError
        Position: Position
          Index: 0
          Line: 1
          Column: 1
          StreamName: 
        UserState: <null>
        Messages: ErrorMessageList
          Head: ExpectedString
            String: #!
            Type: ExpectedString
          Tail: <null>
      Item3: <null>


In [ ]:
let content =
    (newline >>. magicMarker) <|> (eof >>. preturn "")
    |> attempt
    |> lookAhead
    |> manyTill anyChar
    |>> (String.Concat >> String.trim)

In [ ]:
//// test

"""#!magic


a


"""
|> run content
|> _equal (
    Success ("""#!magic


a""", (), Position ("", 14, 7, 1))
)

Success
      Item1: #!magic


a
      Item2: <null>
      Item3: Position
        Index: 14
        Line: 7
        Column: 1
        StreamName: 


In [ ]:
let block =
    pipe2
        magicCommand
        content
        (fun magic content ->
            {
                magic = magic
                content = content
            })

In [ ]:
//// test

"""#!magic


a


"""
|> run block
|> _equal (
    Success (
        { magic = "magic"; content = "a" },
        (),
        Position ("", 14, 7, 1)
    )
)

Success
      Item1: Block
        magic: magic
        content: a
      Item2: <null>
      Item3: Position
        Index: 14
        Line: 7
        Column: 1
        StreamName: 


In [ ]:
let blocks =
    skipMany newline
    >>. sepEndBy block (skipMany1 newline)

In [ ]:
//// test


"""#!magic1

a

\#!magic2

b

"""
|> escapeCell
|> run blocks
|> _equal (
    Success (
        [
            { magic = "magic1"; content = "a" }
            { magic = "magic2"; content = "b" }
        ],
        (),
        Position ("", 26, 9, 1)
    )
)

Success
      Item1: FSharpList<Block>
        - magic: magic1
          content: a
        - magic: magic2
          content: b
      Item2: <null>
      Item3: Position
        Index: 26
        Line: 9
        Column: 1
        StreamName: 


In [ ]:
let inline formatBlock kernel (block : Block) =
    match kernel, block with
    | _, { magic = "markdown"; content = content } ->
        content
        |> String.split [| '\n' |]
        |> Array.map (String.trimEnd [||])
        |> Array.filter (String.endsWith " (test)" >> not)
        |> Array.map (function
            | "" -> "///"
            | line -> System.Text.RegularExpressions.Regex.Replace (line, "^\\s*", "$&/// ")
        )
        |> String.concat "\n"
    | "fsharp", { magic = "fsharp"; content = content } ->
        let trimmedContent = content |> String.trim
        if trimmedContent |> String.startsWith "//// test" || trimmedContent |> String.startsWith "//// ignore"
        then ""
        else
            content
            |> String.split [| '\n' |]
            |> Array.filter (String.trimStart [||] >> String.startsWith "#r" >> not)
            |> String.concat "\n"
    | _ -> ""

In [ ]:
//// test

"""#!markdown


a

    b

c


\#!markdown


c


\#!fsharp


let a = 1"""
|> escapeCell
|> run block
|> function
    | Success (block, _, _) -> formatBlock "fsharp" block
    | Failure (msg, _, _) -> failwith msg
|> _equal """/// a
///
    /// b
///
/// c"""

/// a
///
    /// b
///
/// c


In [ ]:
let inline formatBlocks kernel blocks =
    blocks
    |> List.map (formatBlock kernel)
    |> List.filter ((<>) "")
    |> String.concat "\n\n"
    |> fun s -> s + "\n"

In [ ]:
//// test

"""#!markdown


a

b


\#!markdown


c


\#!fsharp


let a = 1

\#!markdown

d (test)

\#!fsharp

//// test

let a = 2

\#!markdown

e

\#!fsharp

let a = 3"""
|> escapeCell
|> run blocks
|> function
    | Success (blocks, _, _) -> formatBlocks "fsharp" blocks
    | Failure (msg, _, _) -> failwith msg
|> _equal """/// a
///
/// b

/// c

let a = 1

/// e

let a = 3
"""

/// a
///
/// b

/// c

let a = 1

/// e

let a = 3



In [ ]:
let inline parse kernel input =
    match run blocks input with
    | Success (blocks, _, _) ->
        let inline indentBlock (block : Block) =
            { block with
                content =
                    block.content
                    |> String.split [| '\n' |]
                    |> Array.map (fun line ->
                        if line |> String.trimEnd [||] = ""
                        then ""
                        else $"    {line}"
                    )
                    |> String.concat "\n"
            }

        let blocks = blocks |> List.filter (fun block -> block.magic = kernel || block.magic = "markdown")
        
        match blocks with
        | { magic = "markdown"; content = content } :: _
            when kernel = "fsharp"
            && content |> String.startsWith "# "
            && content |> String.endsWith ")"
            ->
            let moduleName, namespaceName =
                System.Text.RegularExpressions.Regex.Match (content, @"# (.*) \((.*)\)$")
                |> fun m -> m.Groups.[1].Value, m.Groups.[2].Value

            let moduleBlock =
                {
                    magic = "fsharp"
                    content =
                        $"""#if !INTERACTIVE
namespace {namespaceName}
#endif

module {moduleName} ="""
                }

            blocks
            |> List.indexed
            |> List.fold
                (fun blocks (index, block) ->
                    match index with
                    | 0 -> blocks
                    | 1 -> indentBlock block :: moduleBlock :: blocks
                    | _ -> indentBlock block :: blocks
                )
                []
            |> List.rev
        | _ -> blocks
        |> Result.Ok
    | Failure (errorMsg, _, _) -> Result.Error errorMsg

In [ ]:
//// test

let example1 =
    """#!meta

{"kernelInfo":{"defaultKernelName":"fsharp","items":[{"aliases":[],"name":"fsharp"},{"aliases":[],"name":"fsharp"}]}}

\#!markdown

# TestModule (TestNamespace)

\#!fsharp

\#!import file.dib

\#!fsharp

\#r "nuget:Expecto"

\#!markdown

## ParserLibrary

\#!fsharp

open System

\#!markdown

## x (test)

\#!fsharp

//// ignore

let x = 1

\#!markdown

### TextInput

\#!fsharp

type Position =
    {
        line : int
        column : int
    }"""
    |> escapeCell

In [ ]:
//// test

example1
|> parse "fsharp"
|> Result.toOption
|> Option.get
|> (formatBlocks "fsharp")
|> _equal """#if !INTERACTIVE
namespace TestNamespace
#endif

module TestModule =

    /// ## ParserLibrary

    open System

    /// ### TextInput

    type Position =
        {
            line : int
            column : int
        }
"""

#if !INTERACTIVE
namespace TestNamespace
#endif

module TestModule =

    /// ## ParserLibrary

    open System

    /// ### TextInput

    type Position =
        {
            line : int
            column : int
        }



In [ ]:
//// test

example1
|> parse "markdown"
|> Result.toOption
|> Option.get
|> (formatBlocks "markdown")
|> _equal """/// # TestModule (TestNamespace)

/// ## ParserLibrary

/// ### TextInput
"""

/// # TestModule (TestNamespace)

/// ## ParserLibrary

/// ### TextInput



In [ ]:
let inline parseDibCode kernel file = async {
    let getLocals () = $"kernel: {kernel} / file: {file} / {getLocals ()}"
    trace Debug (fun () -> "parseDibCode") getLocals
    let! input = File.ReadAllTextAsync file |> Async.AwaitTask
    match parse kernel input with
    | Result.Ok blocks -> return blocks |> formatBlocks kernel
    | Result.Error msg -> return failwith msg
}

In [ ]:
let inline writeDibCode kernel file = async {
    let getLocals () = $"kernel: {kernel} / file: {file} / {getLocals ()}"
    trace Debug (fun () -> "writeDibCode") getLocals
    let! output = parseDibCode kernel file
    let outputFileName =
        match kernel with
        | "fsharp" -> file |> String.replace ".dib" ".fs"
        | _ -> failwith "Unknown kernel"
    do! File.WriteAllTextAsync (outputFileName, output) |> Async.AwaitTask
}

In [ ]:
//// test

let paths =
    match Environment.GetEnvironmentVariable "OUTPUT" with
    | "" | null -> [||]
    | path when System.IO.File.Exists path -> [| path |]
    | path when path.Contains ";" -> path |> String.split [| ';' |]
    | _ -> [| System.IO.Path.Combine (System.IO.Directory.GetCurrentDirectory (), "DibParser.dib") |]

paths
|> Array.map (writeDibCode "fsharp")
|> Async.Parallel
|> Async.Ignore
|> Async.RunSynchronously

21:35:16 #1 [Debug] writeDibCode / kernel: fsharp / file: JsonParser.dib
21:35:16 #1 [Debug] writeDibCode / kernel: fsharp / file: Parser.dib
21:35:16 #1 [Debug] writeDibCode / kernel: fsharp / file: DibParser.dib
21:35:16 #4 [Debug] parseDibCode / kernel: fsharp / file: DibParser.dib
21:35:16 #4 [Debug] parseDibCode / kernel: fsharp / file: JsonParser.dib
21:35:16 #4 [Debug] parseDibCode / kernel: fsharp / file: Parser.dib
